In [1]:
import sys
sys.version

'3.5.2 |Anaconda 4.1.1 (x86_64)| (default, Jul  2 2016, 17:52:12) \n[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]'

In [2]:
import justext
import requests
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.probability import FreqDist
from heapq import nlargest
from bs4 import BeautifulSoup

In [3]:
url = 'http://blog.aurelia.io/2016/12/08/big-aurelia-release-update/'
response = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})

In [4]:
# response.content

In [5]:
soup = BeautifulSoup(response.content, 'html.parser')

In [6]:
metas = soup.find_all('meta')

In [7]:
meta_summary = ''
for meta in metas:
    try:
        if meta['property'] == 'og:description':
            meta_summary = meta['content']
    except:
        pass
print(meta_summary)

We've had a huge set of releases this week, including everything from bug fixes, to performance improvements, new features and fundamentals work that prepares the way for what's next. The full details are below... New Releases aurelia-hot-module-reload Have you ever wished that you could make an HTML, CSS or JavaScript


In [8]:
sent_tokenize(meta_summary)

["We've had a huge set of releases this week, including everything from bug fixes, to performance improvements, new features and fundamentals work that prepares the way for what's next.",
 'The full details are below... New Releases aurelia-hot-module-reload Have you ever wished that you could make an HTML, CSS or JavaScript']

In [9]:
sent_meta_summary = sent_tokenize(meta_summary)
if len(sent_meta_summary) > 1:
    del sent_meta_summary[-1]
meta_summary = '\n'.join(sent_meta_summary)
print(meta_summary)

We've had a huge set of releases this week, including everything from bug fixes, to performance improvements, new features and fundamentals work that prepares the way for what's next.


In [10]:
paragraphs = justext.justext(response.content, justext.get_stoplist("English"))
text = ""
for paragraph in paragraphs:
    if not paragraph.is_boilerplate and not paragraph.is_heading:
        text += paragraph.text

In [11]:
sents = sent_tokenize(text)

In [12]:
word_sent = word_tokenize(text.lower())

In [13]:
_stopwords = set(stopwords.words('english') + list(punctuation) + list(["—", "``", "''"]))

In [14]:
word_sent=[word for word in word_sent if word not in _stopwords]

In [15]:
freq = FreqDist(word_sent)

In [16]:
nlargest(10, freq, key=freq.get)

['changes',
 'new',
 'pal',
 'enables',
 'run',
 'aurelia',
 'release',
 "'s",
 'text',
 'fixes']

In [17]:
from collections import defaultdict
ranking = defaultdict(int)

for i,sent in enumerate(sents):
    for w in word_tokenize(sent.lower()):
        if w in freq:
            ranking[i] += freq[w]

In [18]:
sents_idx = nlargest(4, ranking, key=ranking.get)

In [19]:
summary = meta_summary
summary_sents = [sents[j] for j in sorted(sents_idx)]
if len(summary_sents) > 0:
    summary = [summary] + summary_sents

In [20]:
print(meta_summary)
print(summary_sents[0])

We've had a huge set of releases this week, including everything from bug fixes, to performance improvements, new features and fundamentals work that prepares the way for what's next.
Today, we have our third PAL implementation release, aurelia-pal-worker, which enables various aspects of Aurelia to run inside of a web worker.Major updates may include breaking changes, new features, fixes to bugs and performance improvements.
